In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
from matplotlib.font_manager import fontManager, FontProperties

path = "/usr/share/fonts/opentype/linux-libertine/LinBiolinum_R.otf"
fontManager.addfont(path)
sns.set_theme(style="ticks", font=FontProperties(fname=path).get_name(), font_scale=1.2)

In [ ]:
data = pd.concat([pd.read_csv(data, names=[
    "Variant", "#Replica/Region", "#Client/Region", "#PutClient/Region", "Throughput", "Latency"
]) for data in glob('cops-*.txt')], ignore_index=True)
data["Put %"] = data["#PutClient/Region"] / 2 / data["#Client/Region"] * 100
data = data.sort_values(by="Latency")
data["Latency (ms)"] = data["Latency"] * 1000
data

In [ ]:
f = sns.lineplot(data=data[data["Put %"] == 20],
    x="Throughput", y="Latency (ms)", sort=False,
    hue="Variant", style="Variant", markers=['o'], linewidth=2.5,
    hue_order=["Untrusted", "Replicated (local)", "Quorum"])
# sns.move_legend(f, "upper left")
# f.set(xlim=(0, 110000), ylim=(0, 0.013))
f.set(xlim=(0, 110000), ylim=(0.5, 600), yscale="log")
f.axhline(450, color="red", linestyle="--")
f.set(xticklabels=["0", "20K", "40K", "60K", "80K", "100K"])
f.set(xlabel="Thoughput (op/sec)")

f.figure.savefig("throughput-latency.pdf", bbox_inches='tight')

In [ ]:
f = sns.lineplot(data=data.groupby(["Put %", "Variant"]).max(),
    x="Put %", y="Throughput", hue="Variant", style="Variant", markers=['o'],
    hue_order=["Untrusted", "Replicated (local)", "Quorum"])
f.set(ylim=(0, 380000))
f.set(yticklabels=[0] + [f"{x}K" for x in range(50, 351, 50)])
f.figure.savefig("ratio.pdf", bbox_inches='tight')
